In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('kerala_hospitals.csv')
def show(df):
    return df.head(3)

show(df)

,Name,Location,Rating,coord
0,A J Hospital,Trivandrum,3.0,"8.568439896302305, 76.87292385739063"
1,A M Hospital,Karunagappally,3.4,"9.057598930436313, 76.535243308784"
2,A P Varkey Mission Hospital,Arakunnam,3.4,"9.887120847757341, 76.44717776853612"


In [3]:
df.dropna(subset=['Location'], inplace=True)
df.isna().sum()

Name        0
Location    0
Rating      0
coord       0
dtype: int64

In [4]:
df['Location'].value_counts()

Kochi              30
Trivandrum         23
Kozhikode          14
Thrissur           13
Aluva Market        8
Palakkad            7
Kollam              7
Kottayam            6
Angamaly            6
Thiruvalla          5
Kasaragod           4
Nilambur            4
North Paravur       3
Irinjalakuda        3
Perinthalmanna      3
Cherthala           3
Thodupuzha          3
kalpetta            3
Kanhangad           2
Kodungallur         2
Manjeri             2
Kottakkal           2
Thalassery          2
Perumbavoor         2
Pattambi            2
Taliparamba         2
Karunagappally      2
Kayamkulam          2
Pathanamthitta      2
Chengannur          2
Chalakudy           2
Malappuram          2
Ottapalam           1
Kannur              1
Punalur             1
Chavakkad           1
Alappuzha           1
Parassala           1
Muvattupuzha        1
Chelakkara          1
Kangazha            1
Kolenchery          1
Nedumkandam         1
Anjarakandy         1
Kallambalam         1
Arakunnam 

In [5]:
lat = []
lon = []
for coord in df['coord']:
    latitude, longitude = coord.split(',')
    lat.append(latitude)
    lon.append(longitude)

In [6]:
df['Lat'] = lat
df['Lon'] = lon
df.drop(columns='coord', inplace=True)
show(df)

,Name,Location,Rating,Lat,Lon
0,A J Hospital,Trivandrum,3.0,8.568439896302305,76.87292385739063
1,A M Hospital,Karunagappally,3.4,9.057598930436313,76.535243308784
2,A P Varkey Mission Hospital,Arakunnam,3.4,9.887120847757341,76.44717776853612


In [7]:
df.dtypes

Name         object
Location     object
Rating      float64
Lat          object
Lon          object
dtype: object

In [8]:
df['Lat'] = pd.to_numeric(df['Lat'])
df['Lon'] = pd.to_numeric(df['Lon'])
df.dtypes

Name         object
Location     object
Rating      float64
Lat         float64
Lon         float64
dtype: object

Getting the average rating within areas

In [9]:
avg_rating = df.groupby(['Location'])['Rating'].mean().to_frame().reset_index()
show(avg_rating)

,Location,Rating
0,Alappuzha,3.300000
1,Aluva Market,3.487500
2,Angamaly,3.533333


In [10]:
# Getting coordinates of the regions 
lat=[]
lon=[]
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="app")

for location in avg_rating['Location']:
    coord = geolocator.geocode(location)
    if coord is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(coord.latitude)
        lon.append(coord.longitude)

In [11]:
avg_rating['Lat'] = lat
avg_rating['Lon'] = lon
show(avg_rating)

,Location,Rating,Lat,Lon
0,Alappuzha,3.300000,9.500665,76.412414
1,Aluva Market,3.487500,10.106761,76.348840
2,Angamaly,3.533333,10.191028,76.387411


In [12]:
'''
The geocoding of 'Aluva' wasn't correct. Hence, it was replaced 'Aluva Market' in the csv file. 
After acquiring the correct location, the original name is re-assigned. 
'''
dict = {'Aluva Market':'Aluva'}
avg_rating['Location'] = avg_rating['Location'].replace(dict)

In [13]:
# Let's make the map
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import json
from folium import plugins



In [15]:
basemap=folium.Map(location=[10.5,76],zoom_start=8, width='100', tiles='openstreetmap') #"Stamen Toner, cartodbpositron"

# Marker cluster for hospital count
mc = MarkerCluster(name='Hospital count')
for idx, row in df.iterrows():
    mc.add_child(folium.Marker([row['Lat'], row['Lon']],popup=[row['Name'], row['Rating']]))
basemap.add_child(mc)

# heat map for displaying average rating
HeatMap(avg_rating[['Lat', 'Lon', 'Rating']], name='Average rating', radius=17, min_opacity=0.4).add_to(basemap)

# A mini map to show the AOI
mini_map = plugins.MiniMap(toggle_display=True, position='bottomleft',zoom_level_offset=-6, tile_layer='Stamen Toner')
basemap.add_child(mini_map)

# Various basemaps
folium.TileLayer('OpenStreetMap').add_to(basemap)
folium.TileLayer('cartodbpositron').add_to(basemap)
folium.TileLayer('Stamen Toner').add_to(basemap)
folium.LayerControl().add_to(basemap)

# Display the map
basemap

In [16]:
basemap.save('Region-wise_Hospital_rating_Kerala.html')

<h2>Regional Hospital Rating</h2>

Markers in this interactive map represents the number of hospitals in each region. When zooming in, the areas are separated into sub-regions until individual markers identified. While clicking on a specific marker, it will display the name of the hospital and its public rating out of five. 
The heatmap displays average rating of all hospitals within each region. A higher grade is indicated by a color code ranging from blue to red.
<br>
<i> *Findings are given below the map</i><br>



<h3>Findings </h3>
1. Considering both the number of hospitals and their rating, the region from Kochi to Irinjalakuda exhibits best result.<br>
2. There are significantly less hospitals in northern Kerala (Malabar area).<br>
3. There is a considerable lack of hospitals between Kozhikode and Kannur, as well as in the hilly areas of south Kerala.<br>
4. Hospitals in Malappuram district have highest public rating.<br>
5. The number of hospitals and their public rating are reduced as moving away from coastal regions.<br>
6. Despite the fact that Trivandrum has the most hospitals (23) after Kochi (30), the average public rating is lower.<br>

<h3> Disclaimers </h3>
1. The list of hospitals is obtained from https://www.medicineindia.org/hospitals-in-state/kerala. The data is unofficial and unverfied. <br>
2. Hospitals with no rating or a rating with less number of voting are discarded.<br>
3. Locations obtained using geocode function is not verified.<br>
4. The results are unofficial and not verified.